Random Forests: 0.64

In [1]:
import numpy as np
import pandas as pd
from sklearn import svm
from sklearn import metrics
from sklearn.metrics import f1_score
from matplotlib import pyplot as plt
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.ensemble import BaggingClassifier
from sklearn.preprocessing import RobustScaler
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split


In [2]:
dataPath = '../../preprocess_data_set/newsData_ratio8_2.csv'

df_train = pd.read_csv(dataPath)
df_train.shape

(26771, 251)

In [3]:
# randomly shuffle the rows
df_train = df_train.sample(frac=1, random_state=42)

df_train = df_train.reset_index()
df_train = df_train.drop('index', axis=1)
df_train

,0,1,2,3,4,5,6,7,8,9,...,241,242,243,244,245,246,247,248,249,target
0,19.657124,50.442482,20.871874,-54.531494,20.362791,28.577585,-1.906657,28.021065,-24.411823,-19.387388,...,-49.060287,-3.104522,27.967051,-30.539959,0.434239,36.554264,-1.757592,37.968136,-14.561114,0
1,73.660179,76.221085,-40.807030,-82.814301,28.292801,36.778252,-1.909100,49.479527,-79.806595,27.742567,...,-107.097870,-24.814592,-22.166080,3.848902,36.787331,59.989563,-6.423959,21.630405,-88.683472,0
2,-13.957203,50.162762,60.824032,-16.521429,-84.106873,33.965801,-24.306961,112.798645,-90.984291,-23.109505,...,-37.652954,39.876980,101.627777,-16.215570,-2.601095,74.666214,-34.334938,4.400374,-16.785315,1
3,34.285301,24.369213,37.394283,-56.808426,-13.195957,27.064581,-27.190607,45.300884,-38.357361,-21.576128,...,-37.443409,8.713444,20.008799,-13.551638,16.853464,48.484596,-13.077551,5.347030,-43.575981,0
4,8.285158,182.228394,3.545697,-74.310005,-46.353405,27.703939,-23.581633,136.181381,-75.572617,-49.096653,...,-108.719910,51.889614,156.074570,9.806570,61.040123,24.597773,-46.054447,-51.063011,-56.279831,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26766,-71.454414,56.266552,-52.882614,-45.946815,-243.017334,76.435471,-80.246399,212.383789,-183.680710,-156.889847,...,-57.277454,78.127159,146.414536,-44.106407,59.677574,120.351700,-139.349457,35.773731,6.932467,1
26767,-0.284165,25.422646,2.497849,-8.947376,-36.106369,29.048079,-8.172162,28.610073,-23.188131,-27.730436,...,4.322698,22.755190,8.053110,2.871661,8.147329,12.612817,-6.104894,10.478649,2.183957,0
26768,71.339561,16.005495,51.056477,-179.901749,-14.259531,126.382141,-4.463017,42.901329,53.434349,-14.907699,...,-106.613243,-39.113091,48.542309,-67.660194,58.561333,206.557541,-90.538963,154.817764,-92.831696,0
26769,81.606827,-113.467224,-3.810923,-238.568436,70.856094,87.432976,53.989868,111.472473,-73.990349,42.114433,...,-40.093098,107.691345,-5.296010,-85.728683,78.135361,180.972107,-158.033691,8.803612,-97.057518,0


In [4]:
train_y = df_train['target']
train_x = df_train.drop(columns=['target'])

In [5]:
# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(train_x, train_y, test_size=0.3,random_state=109) # 70% training and 30% test

In [6]:
robust_scaler = RobustScaler().fit(X_train.values)
trans_x = robust_scaler.transform(X_train.values)

In [7]:
trans_x.shape

(18739, 250)

In [8]:
# Create a pipeline with PCA and D-forests
# higher principle components and estimators increase the accuracy as well as the model construction time
# current it takes 2 minutes to construct the model 
pc_num = 100

pipe = Pipeline([
    ('pca', PCA(n_components = pc_num)), # choose number of components to keep
    ('clf', RandomForestClassifier(n_estimators = 300))
])

pipe.fit(trans_x, y_train)

Pipeline(steps=[('pca', PCA(n_components=100)),
                ('clf', RandomForestClassifier(n_estimators=300))])

In [9]:
y_pred = pipe.predict(X_test)

c:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:432: UserWarning: X has feature names, but PCA was fitted without feature names
  warnings.warn(


In [12]:
print(metrics.classification_report(y_pred, y_test))

              precision    recall  f1-score   support

           0       0.14      0.91      0.25      1025
           1       0.94      0.21      0.35      7007

    accuracy                           0.30      8032
   macro avg       0.54      0.56      0.30      8032
weighted avg       0.84      0.30      0.33      8032



In [11]:
from sklearn.model_selection import KFold, cross_val_score

k_folds = KFold(n_splits = 5)

pc_num = 100

pipe_cv = Pipeline([
    ('pca', PCA(n_components = pc_num)), # choose number of components to keep
    ('clf', RandomForestClassifier(n_estimators = 300))
])

scores = cross_val_score(pipe_cv, train_x, train_y, cv = k_folds)

print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())

Cross Validation Scores:  [0.95182073 0.94938364 0.94863653 0.94583489 0.94994397]
Average CV Score:  0.9491239514109715
